In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, f1_score
import lightgbm as lgb
from collections import Counter
import warnings
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"  #不然会崩内核
warnings.filterwarnings("ignore")

In [2]:
#!pip --default-timeout=1000 install --index-url https://mirrors.aliyun.com/pypi/simple tensorflow-cpu

In [3]:
#取出某一个时间片的特征（速度，eta速度，状态，车辆数）
def get_base_info(x):
    return [i.split(':')[-1] for i in x.split(' ')]
#取出速度
def get_speed(x):
    return np.array([i.split(',')[0] for i in x], dtype='float16')
#取出eta速度
def get_eta(x):
    return np.array([i.split(',')[1] for i in x], dtype='float16')
#取出状态（畅通，缓行，拥堵）1,2,3,4 ----3,4官方说可以看成一个 hhh
def get_state(x):
    return [int(i.split(',')[2]) for i in x]
#取出通过的车辆数量
def get_cnt(x):
    return np.array([i.split(',')[3] for i in x], dtype='int16')
#对训练集或测试集进行处理
def gen_feats(path, mode='is_train'):
    df = pd.read_csv(path, sep=';', header=None)#sep分隔符，以；分割
    df['link'] = df[0].apply(lambda x: x.split(' ')[0])
    if mode == 'is_train':
        df['label'] = df[0].apply(lambda x: int(x.split(' ')[1]))
        df['label'] = df['label'].apply(lambda x: 3 if x > 3 else x)
        df['label'] -= 1#标签从1，2，3变成0，1，2
        df['current_slice_id'] = df[0].apply(lambda x: int(x.split(' ')[2]))
        df['future_slice_id'] = df[0].apply(lambda x: int(x.split(' ')[3]))
    else:
        df['label'] = -1
        df['current_slice_id'] = df[0].apply(lambda x: int(x.split(' ')[2]))
        df['future_slice_id'] = df[0].apply(lambda x: int(x.split(' ')[3]))

    df['time_diff'] = df['future_slice_id'] - df['current_slice_id']
    df['curr_state'] = df[1].apply(lambda x: x.split(' ')[-1].split(':')[-1])#当前时间片的特征
    df['curr_speed'] = df['curr_state'].apply(lambda x: x.split(',')[0])
    df['curr_eta'] = df['curr_state'].apply(lambda x: x.split(',')[1])
    df['curr_cnt'] = df['curr_state'].apply(lambda x: x.split(',')[3])
    df['curr_state'] = df['curr_state'].apply(lambda x: x.split(',')[2])#当前时间片的状态label
    del df[0]

    for i in tqdm(range(1, 6)):#tqdm 显示一个加载进度条
        df['his_info'] = df[i].apply(get_base_info)
        if i == 1:
            flg = 'current'
        else:
            flg = f'his_{(6 - i) * 7}'
        #提取每一段时间片的数据特征
        df['his_speed'] = df['his_info'].apply(get_speed)
        df[f'{flg}_speed_min'] = df['his_speed'].apply(lambda x: x.min())
        df[f'{flg}_speed_max'] = df['his_speed'].apply(lambda x: x.max())
        df[f'{flg}_speed_mean'] = df['his_speed'].apply(lambda x: x.mean())
        df[f'{flg}_speed_std'] = df['his_speed'].apply(lambda x: x.std())

        df['his_eta'] = df['his_info'].apply(get_eta)
        df[f'{flg}_eta_min'] = df['his_eta'].apply(lambda x: x.min())
        df[f'{flg}_eta_max'] = df['his_eta'].apply(lambda x: x.max())
        df[f'{flg}_eta_mean'] = df['his_eta'].apply(lambda x: x.mean())
        df[f'{flg}_eta_std'] = df['his_eta'].apply(lambda x: x.std())

        df['his_cnt'] = df['his_info'].apply(get_cnt)
        df[f'{flg}_cnt_min'] = df['his_cnt'].apply(lambda x: x.min())
        df[f'{flg}_cnt_max'] = df['his_cnt'].apply(lambda x: x.max())
        df[f'{flg}_cnt_mean'] = df['his_cnt'].apply(lambda x: x.mean())
        df[f'{flg}_cnt_std'] = df['his_cnt'].apply(lambda x: x.std())

        df['his_state'] = df['his_info'].apply(get_state)
        #counter（）函数返回的是一个类似于字典的counter计数器
        #Counter类中的most_common(n)函数:传进去一个可选参数n(代表获取数量最多的前n个元素，如果不传参数，代表返回所有结果)
        df[f'{flg}_state'] = df['his_state'].apply(lambda x: Counter(x).most_common()[0][0])
        df.drop([i, 'his_info', 'his_speed', 'his_eta', 'his_cnt', 'his_state'], axis=1, inplace=True)
    if mode == 'is_train':
        r='\\'
        df.to_csv(f"__{mode}_{path.split(r)[-1]}", index=False)
    else:
        df.to_csv(f"is_test.csv", index=False)

In [4]:
#算法评价指标
def f1_score_eval(preds, valid_df):
    labels = valid_df.get_label()
    preds = np.argmax(preds.reshape(3, -1), axis=0)
    scores = f1_score(y_true=labels, y_pred=preds, average=None)
    scores = scores[0]*0.2+scores[1]*0.2+scores[2]*0.6
    return 'f1_score', scores, True

In [5]:
#gen_feats(r"D:\RoadStatusData\traffic\20190701.txt", mode='is_train')

In [6]:
def lgb_train(train_: pd.DataFrame, test_: pd.DataFrame, use_train_feats: list, id_col: str, label: str,
              n_splits: int, split_rs: int, is_shuffle=True, use_cart=False, cate_cols=None) -> pd.DataFrame:
    if not cate_cols:
        cate_cols = []
    print('data shape:\ntrain--{}\ntest--{}'.format(train_.shape, test_.shape))#数据维度
    print('Use {} features ...'.format(len(use_train_feats)))#有几个特征
    print('Use lightgbm to train ...')
    n_class = train_[label].nunique()#unique()是以 数组形式（numpy.ndarray）返回列的所有唯一值（特征的所有唯一值）
                                     #nunique() 返回的是唯一值的个数
                                     #这里n_class代表有几个分类，1，2，3即三种
    train_[f'{label}_pred'] = 0
    test_pred = np.zeros((test_.shape[0], n_class))#预测先置为0
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = use_train_feats

    folds = KFold(n_splits=n_splits, shuffle=is_shuffle, random_state=split_rs)#n_split:要划分的折数
                                                                               #shuffle: 每次都进行shuffle，测试集中折数的总和就是训练集的个数
                                                                               #random_state:随机状态
    train_user_id = train_[id_col].unique()#返回所有id的唯一值

    #模型参数设置
    params = {
        'learning_rate': 0.05,#学习率
        'boosting_type': 'gbdt', #gbdt模型为基础
        'objective': 'multiclass',#多分类
        'metric': 'None',
        'num_leaves': 31,#单棵树的最大叶子数
        'num_class': n_class,#共有多少类
        'feature_fraction': 0.8,# 如果 feature_fraction 小于 1.0, LightGBM 将会在每次迭代中随机选择部分特征. 例如, 如果设置为 0.8, 将会在每棵树训练之前选择 80% 的特征
                                # 可以用来加速训练
                                # 可以用来处理过拟合
        'bagging_fraction': 0.8,# 类似于 feature_fraction, 但是它将在不进行重采样的情况下随机选择部分数据
                                # 可以用来加速训练
                                # 可以用来处理过拟合
                                # Note: 为了启用 bagging, bagging_freq 应该设置为非零值
        'bagging_freq': 5,#bagging 的频率, 0 意味着禁用 bagging. k 意味着每 k 次迭代执行bagging
                          #Note: 为了启用 bagging, bagging_fraction 设置适当
        'seed': 1,
        'bagging_seed': 1,
        'feature_fraction_seed': 7,
        'min_data_in_leaf': 20,#一个叶子上数据的最小数量. 可以用来处理过拟合
        'nthread': -1,
        'verbose': -1
    }

    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_user_id), start=1):#把数据分成几折
        #分成训练集和测试集的下标
        print('the {} training start ...'.format(n_fold))
        train_x, train_y = train_.loc[train_[id_col].isin(train_user_id[train_idx]), use_train_feats], train_.loc[
            train_[id_col].isin(train_user_id[train_idx]), label]#bool索引  拆分成特征和标签
        valid_x, valid_y = train_.loc[train_[id_col].isin(train_user_id[valid_idx]), use_train_feats], train_.loc[
            train_[id_col].isin(train_user_id[valid_idx]), label]#被分成测试集的特征和标签
        print(f'for train user:{len(train_idx)}\nfor valid user:{len(valid_idx)}')

        if use_cart:
            dtrain = lgb.Dataset(train_x, label=train_y, categorical_feature=cate_cols)
            #lightgbm可以处理标称型（类别）数据。通过指定'categorical_feature' 这一参数告诉它哪些feature是标称型的。
            # 它不需要将数据展开成独热码(one-hot)，其原理是对特征的所有取值，做一个one-vs-others，从而找出最佳分割的那一个特征取值
            dvalid = lgb.Dataset(valid_x, label=valid_y, categorical_feature=cate_cols)
        else:
            dtrain = lgb.Dataset(train_x, label=train_y)
            dvalid = lgb.Dataset(valid_x, label=valid_y)
            
        #训练
        clf = lgb.train(
            params=params,#模型参数
            train_set=dtrain,#训练集
            num_boost_round=5000,#迭代次数
            valid_sets=[dvalid],#测试集
            early_stopping_rounds=100,#如果一次验证数据的一个度量在最近的early_stopping_round 回合中没有提高，模型将停止训练 加速分析，减少过多迭代
            verbose_eval=100,#迭代几次传回评估结果---正确率召回率指标
            feval=f1_score_eval#评价函数
        )
         #  统计某种特征在整个.py文件中使用的次数
        fold_importance_df[f'fold_{n_fold}_imp'] = clf.feature_importance(importance_type='gain')#特征重要度 哈哈哈
        train_.loc[train_[id_col].isin(train_user_id[valid_idx]), f'{label}_pred'] = np.argmax(
            clf.predict(valid_x, num_iteration=clf.best_iteration), axis=1)
        test_pred += clf.predict(test_[use_train_feats], num_iteration=clf.best_iteration) / folds.n_splits

    report = f1_score(train_[label], train_[f'{label}_pred'], average=None)#计算分数
    print(classification_report(train_[label], train_[f'{label}_pred'], digits=4))#评估指标 4个小数
    print('Score: ', report[0] * 0.2 + report[1] * 0.2 + report[2] * 0.6)
    test_[f'{label}_pred'] = np.argmax(test_pred, axis=1)
    test_[label] = np.argmax(test_pred, axis=1)+1 #测试集标签
    
    # test_[label] = np.argmax(test_pred, axis=1)
    #统计数据
    five_folds = [f'fold_{f}_imp' for f in range(1, n_splits + 1)]
    fold_importance_df['avg_imp'] = fold_importance_df[five_folds].mean(axis=1)
    fold_importance_df.sort_values(by='avg_imp', ascending=False, inplace=True)
    print(fold_importance_df[['Feature', 'avg_imp']].head(20))
    return test_[[id_col, 'current_slice_id', 'future_slice_id', label]]

In [7]:
#将后6天的训练集合并  失败
'''
df1=pd.read_csv(r"D:\RoadStatusData\traffic\20190725.txt")
for i in range(26,31):
    df2=pd.read_csv(f"D:\\RoadStatusData\\traffic\\201907{i}.txt")
    df1=pd.concat([df1,df2])
df1.to_csv(r"merge_25_30.txt", index=False, encoding='utf8')'''
#采用写入的方式
'''for i in tqdm(range(25,31)):
    with open(f'D:\\RoadStatusData\\traffic\\201907{i}.txt', 'r') as text:
      with open(r'D:\RoadStatusData\traffic\merge_25_30.txt', 'a') as txt:
        txt.writelines(text.readlines())'''

"for i in tqdm(range(25,31)):\n    with open(f'D:\\RoadStatusData\\traffic\\201907{i}.txt', 'r') as text:\n      with open(r'D:\\RoadStatusData\traffic\\merge_25_30.txt', 'a') as txt:\n        txt.writelines(text.readlines())"

In [8]:
if __name__ == "__main__":
    #train_path = r'D:\RoadStatusData\traffic\20190730.txt'
    train_path=r'D:\RoadStatusData\traffic\merge_25_30.txt'
    test_path = r'D:\RoadStatusData\20190801_testdata.txt'
    gen_feats(train_path, mode='is_train')
    gen_feats(test_path, mode='is_test')
    attr = pd.read_csv(r'D:\RoadStatusData\attr.txt', sep='\t',
                       names=['link', 'length', 'direction', 'path_class', 'speed_class', 'LaneNum', 'speed_limit',
                              'level', 'width'], header=None)

    train = pd.read_csv('__is_train_merge_25_30.txt')
    test = pd.read_csv('is_test.csv')
    train = train.merge(attr, on='link', how='left')
    test = test.merge(attr, on='link', how='left')

    use_cols = [i for i in train.columns if i not in ['link', 'label', 'current_slice_id', 'future_slice_id', 'label_pred']]

    sub = lgb_train(train, test, use_cols, 'link', 'label', 5, 2020)

    sub.to_csv('RoadLGBMPre.csv', index=False, encoding='utf8')

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:47<00:00, 21.58s/it]


data shape:
train--(2989065, 82)
test--(176057, 82)
Use 78 features ...
Use lightgbm to train ...
the 1 training start ...
for train user:11276
for valid user:2819
Training until validation scores don't improve for 100 rounds
[100]	valid_0's f1_score: 0.588867
[200]	valid_0's f1_score: 0.591889
[300]	valid_0's f1_score: 0.592115
Early stopping, best iteration is:
[290]	valid_0's f1_score: 0.592398
the 2 training start ...
for train user:11276
for valid user:2819
Training until validation scores don't improve for 100 rounds
[100]	valid_0's f1_score: 0.587643
[200]	valid_0's f1_score: 0.59114
[300]	valid_0's f1_score: 0.591677
Early stopping, best iteration is:
[227]	valid_0's f1_score: 0.591903
the 3 training start ...
for train user:11276
for valid user:2819
Training until validation scores don't improve for 100 rounds
[100]	valid_0's f1_score: 0.602938
[200]	valid_0's f1_score: 0.608115
[300]	valid_0's f1_score: 0.608211
[400]	valid_0's f1_score: 0.608348
Early stopping, best iteratio

In [9]:
'''df=pd.read_csv(r'D:\RoadStatusData\traffic\20190701.txt', sep=';',header=None)
df.head()'''

"df=pd.read_csv(r'D:\\RoadStatusData\traffic\x8190701.txt', sep=';',header=None)\ndf.head()"

In [10]:
'''df['link'] = df[0].apply(lambda x: x.split(' ')[0])
df['label'] = df[0].apply(lambda x: int(x.split(' ')[1]))
df['label'] = df['label'].apply(lambda x: 3 if x > 3 else x)
df['label'] -= 1
df['current_slice_id'] = df[0].apply(lambda x: int(x.split(' ')[2]))
df['future_slice_id'] = df[0].apply(lambda x: int(x.split(' ')[3]))

df['time_diff'] = df['future_slice_id'] - df['current_slice_id']
df['curr_state'] = df[1].apply(lambda x: x.split(' ')[-1].split(':')[-1])#取当前时间片的特征
df['curr_speed'] = df['curr_state'].apply(lambda x: x.split(',')[0])
df['curr_eta'] = df['curr_state'].apply(lambda x: x.split(',')[1])
df['curr_cnt'] = df['curr_state'].apply(lambda x: x.split(',')[3])
df['curr_state'] = df['curr_state'].apply(lambda x: x.split(',')[2])#当前时间片的状态label
del df[0]
df'''

"df['link'] = df[0].apply(lambda x: x.split(' ')[0])\ndf['label'] = df[0].apply(lambda x: int(x.split(' ')[1]))\ndf['label'] = df['label'].apply(lambda x: 3 if x > 3 else x)\ndf['label'] -= 1\ndf['current_slice_id'] = df[0].apply(lambda x: int(x.split(' ')[2]))\ndf['future_slice_id'] = df[0].apply(lambda x: int(x.split(' ')[3]))\n\ndf['time_diff'] = df['future_slice_id'] - df['current_slice_id']\ndf['curr_state'] = df[1].apply(lambda x: x.split(' ')[-1].split(':')[-1])#取当前时间片的特征\ndf['curr_speed'] = df['curr_state'].apply(lambda x: x.split(',')[0])\ndf['curr_eta'] = df['curr_state'].apply(lambda x: x.split(',')[1])\ndf['curr_cnt'] = df['curr_state'].apply(lambda x: x.split(',')[3])\ndf['curr_state'] = df['curr_state'].apply(lambda x: x.split(',')[2])#当前时间片的状态label\ndel df[0]\ndf"